In [1]:
import pandas as pd
import os
import numpy as np
import re

In [218]:
def add_space_after_punctuation(text):
    """Adds a single space after each period and comma in the given text."""
    text = re.sub(r'(?<=[.,])(?=[^\s])', ' ', text)
    return text

def replace_terms(text, search_terms, replacement_terms):
    """
    Replaces substrings in text based on search_terms and replacement_terms,
    avoiding overlaps and prioritizing longer matches first.
    Any spaces in replacement_terms are replaced with underscores in the .md link.
    """
    # Replace spaces with underscores in target filenames
    replacement_terms = [r.replace(' ', '_') for r in replacement_terms]

    # Sort by descending length to prioritize longer matches
    sorted_terms = sorted(zip(search_terms, replacement_terms), key=lambda x: len(x[0]), reverse=True)

    # Build a regex pattern to match any of the search terms
    pattern = re.compile('|'.join(re.escape(term) for term, _ in sorted_terms))

    # Track which spans have already been replaced
    replaced_spans = []
    result = []
    last_index = 0

    for match in pattern.finditer(text):
        start, end = match.span()

        # Check if this match overlaps any previously replaced span
        if any(rs < end and start < re for rs, re in replaced_spans):
            continue

        matched_text = match.group(0)

        # Look up replacement
        for syn, off in sorted_terms:
            if matched_text == syn:
                replacement = f"[{matched_text}]('{off}.md')"
                break
        else:
            replacement = matched_text  # fallback

        # Append previous text + replacement
        result.append(text[last_index:start])
        result.append(replacement)

        replaced_spans.append((start, end))
        last_index = end

    result.append(text[last_index:])
    return ''.join(result)

def bold_first_sentence(text):
    first_period_index = text.find('.')
    if first_period_index != -1:
        return f"**{text[:first_period_index + 1]}**{text[first_period_index + 1:]}"
    else:
        # If there's no period, bold the entire block
        return f"**{text}**"

def bold_until_colon(text):
    first_colon_index = text.find(':')
    if first_colon_index != -1:
        return f"**{text[:first_colon_index + 1]}**{text[first_colon_index + 1:]}"
    else:
        # If there's no colon, bold the entire block
        return f"**{text}**"

def check_for_letters(text):
    return any(char.isalpha() for char in text)

def find_character_after_index(text, char, start_index):
    """
    Finds the first instance of a character in a string after a certain index.

    Args:
        text (str): The string to search in.
        char (str): The character to find.
        start_index (int): The index to start searching from.

    Returns:
        int: The index of the first instance of the character after the start index, or -1 if not found.
    """
    if start_index < 0 or start_index >= len(text):
        return -1

    for i in range(start_index, len(text)):
        if text[i] == char:
            return i
    return -1

def generate_mkdocs_nav(folder_path, output_file="mkdocs_nav.txt"):
    if not os.path.isdir(folder_path):
        raise ValueError(f"Folder does not exist: {folder_path}")

    md_files = [f for f in os.listdir(folder_path) if f.endswith(".md")]

    # Prepare (display_name, filename) tuples
    entries = []
    for filename in md_files:
        name_without_ext = os.path.splitext(filename)[0]
        display_name = name_without_ext.replace("_", " ").capitalize()
        entries.append((display_name, filename))

    # Sort by display_name
    entries.sort(key=lambda x: x[0].lower())

    # Generate nav lines
    nav_lines = [f"- {display_name}: {filename}" for display_name, filename in entries]

    with open(output_file, "w", encoding="utf-8") as f:
        f.write("\n".join(nav_lines))

    print(f"Nav file written to: {output_file}")

# Create hyperlinked text files

In [3]:
# read in texts
texts = pd.read_csv("./parse_text/verified_synonym_matches.csv")

In [4]:
# get file names of all texts
directory = "./texts" 
filenames = os.listdir(directory)

# remove .DS_Store if present
if '.DS_Store' in filenames:
    filenames.remove('.DS_Store')

# create a new folder to store hyperlinked text if doesn't already exist
folder_name = "hyperlinked_text"  
os.makedirs(folder_name, exist_ok=True)

In [5]:
for i in range(len(filenames)):

    if filenames[i][0]!="c":

        file_path = "./texts/"+filenames[i] # get file name
    
        subchap = filenames[i][:3].lstrip('0') # get subchapter
        subchap_title = "## Subchapter " + subchap + "  \n" # get subchapter title header
        pg_num = filenames[i][(filenames[i].index("_")+2):(filenames[i].index("_")+5)].lstrip('0') # get page number
        link = "https://archive.org/details/aztec-herbal-of-1552/page/"+pg_num # get link to display
        full_link = "["+link+"]("+link+")"+"  \n"
        
        # open text
        with open(file_path, 'r') as file:
            text = file.read()
         
        clean_text = text.replace('\n', '') # remove line breaks
        clean_text = add_space_after_punctuation(clean_text) # make sure space in text
        
        texts["no_zero_subchap"] = texts['subchapter'].str.lstrip('0') # strip leading zeros in df
        subchap_syn = texts["spelling"][texts["no_zero_subchap"]==subchap].to_list() # search for synonyms
        subchap_off = texts["official_name"][texts["no_zero_subchap"]==subchap].to_list() # get official names for replacement
        
        # run text through the replacer
        output = replace_terms(clean_text, subchap_syn, subchap_off)+"  "
        
        # bold first sentence
        bold_output = bold_first_sentence(output) + "  \n"
        
        # markdown
        final_md = subchap_title + bold_output + full_link + "\n"
        
        path = folder_name+"/"+subchap+".txt"  # change to your desired path
        
        with open(path, "w", encoding="utf-8") as f:
            f.write(final_md)
            
    elif filenames[i][0]=="c":
        
        file_path = "./texts/"+filenames[i] # get file name

        chap = filenames[i][7:9].lstrip('0') # get chapter
        chap_title = "## Chapter " + chap + "  \n" # get chapter title header
        pg_num = filenames[i][(filenames[i].index("_")+2):(filenames[i].index("_")+5)].lstrip('0') # get page number
        link = "https://archive.org/details/aztec-herbal-of-1552/page/"+pg_num # get link to display
        full_link = "["+link+"]("+link+")"+"  \n"
        
        # open text
        with open(file_path, 'r') as file:
            text = file.read()
         
        clean_text = text.replace('\n', '') # remove line breaks
        clean_text = add_space_after_punctuation(clean_text) # make sure space in text
        
        # bold until colon
        bold_output = bold_until_colon(clean_text) + "  \n"
        
        # markdown
        final_md = chap_title + bold_output + full_link + "\n"
        
        path = folder_name+"/"+chap+".txt"  # change to your desired path
        
        with open(path, "w", encoding="utf-8") as f:
            f.write(final_md)

# Create subchapter and chapter markdown files

In [6]:
# get file names of all texts
directory = "./hyperlinked_text" 
filenames = os.listdir(directory)

# remove .DS_Store if present
if '.DS_Store' in filenames:
    filenames.remove('.DS_Store')

# create a new folder to store chapter documents
folder_name = "chapters"  
os.makedirs(folder_name, exist_ok=True)

# get file names of all assets
asset_dir = "./assets" 
asset_files = os.listdir(asset_dir)

# remove .DS_Store if present
if '.DS_Store' in asset_files:
    asset_files.remove('.DS_Store')

asset_chap_pgs = [] # save available pages with illustrations
asset_chap_inds = [] # save indices in asset files if page is available
for i in range(len(asset_files)):
    asset_id = asset_files[i][asset_files[i].index("_")+1:asset_files[i].index(".")]
    if asset_id[0]=="p":
        pg = asset_id[1:].lstrip("0")
        asset_chap_pgs.append(pg)
        asset_chap_inds.append(i)
    else:
        continue

In [7]:
for i in range(len(filenames)):
    file = filenames[i]
    id_num = file[:file.index(".")] # get chapter/subchapter number
    if not check_for_letters(id_num): # CHAPTERS
        path = directory+"/"+file
        with open(path, 'r') as file:
            text = file.read()
        path = folder_name+"/"+id_num +".md"  # change to your desired path
        with open(path, "w", encoding="utf-8") as f:
            f.write(text)
    elif check_for_letters(id_num): # SUBCHAPTERS
        path = directory+"/"+file
        with open(path, 'r') as file:
            text = file.read()
        start_ind = text.rfind("/")+1 # USE LINK TO FIND PAGE NUMBER
        end_ind = find_character_after_index(text, ")", start_ind)
        page = text[start_ind:end_ind]
        if page in asset_chap_pgs:
            ill_ind = asset_chap_inds[asset_chap_pgs.index(page)] # get index of illustration
            ill_file = asset_files[ill_ind] # get illustration file name
            ill_path = "!["+ill_file+"](assets/"+ill_file+")  " # get text to add to markdown
    
            initials = ill_file[:ill_file.index("_")] # get initials
            attributions = [] # create a list of attributions
            for i in initials:
                if i=="A":
                    attributions.append("Leaf traces by: Alejandra Rougon, UNAM ENES León, México  ")
                elif i=="D":
                    attributions.append("Leaf traces by: Dan Chitwood, Michigan State University, USA  ")
                elif i=="K":
                    attributions.append("Leaf traces by: Kylie DeViller, Acadia University, Canada  ")
                elif i=="M":
                    attributions.append("Leaf traces by: Mariana Ruíz Amaro, UNAM ENES León, México  ")
                elif i=="N":
                    attributions.append("Leaf traces by: Noé García, UNAM ENES León, México  ")
                elif i=="Z":
                    attributions.append("Leaf traces by: Zoë Migicovsky, Acadia University, Canada  ")
            attr = attributions[0] # keep to just one attribution for now

            # remove first line of text
            lines = text.splitlines(True)
            remaining_lines = lines[1:]
            text = "".join(remaining_lines)
            
            # markdown
            final_md = text + "\n" + ill_path + "\n" + attr
    
            path = folder_name+"/"+id_num+"_ill.md"  # change to your desired path, indicate illustrated
            with open(path, "w", encoding="utf-8") as f:
                f.write(final_md)
        else:
            path = folder_name+"/"+id_num+".md"  # change to your desired path, no illustration

            # remove first line of text
            lines = text.splitlines(True)
            remaining_lines = lines[1:]
            text = "".join(remaining_lines)
            
            with open(path, "w", encoding="utf-8") as f:
                f.write(text) # just text
        
        

# Create different sections of pages

In [18]:
# get plants with illustrations
images = pd.read_csv("./illustrations/assets.csv")
images = images.dropna().reset_index()
image_IDs = images["ID"].unique()


In [19]:
# get references to words in text
texts = pd.read_csv("./parse_text/verified_synonym_matches.csv")
text_IDs = texts["ID"].unique()


In [20]:
# create a sections column, to know what section each ID is
sections = []
for i in range(len(texts)):
    curr_ID = texts["ID"].iloc[i]
    if curr_ID in image_IDs:
        sections.append("plant_image")
    else:
        sections.append(texts["type"].iloc[i])
texts["section"] = sections

# Plants with illustrations

In [216]:
# create a new folder to store chapter documents
folder_name = "plants_with_illlustrations"  
os.makedirs(folder_name, exist_ok=True)


### NEED TO COME BACK TO THIS: PLANTS WITH MULTIPLE IMAGES. NEED TO CHANGE THE ASSETS FILE IN ILLUSTRATIONS FOLDER

In [219]:
# for each ID with an illustration
for id in range(len(image_IDs)):

    # ILLUSTRATIONS
    curr_ID = image_IDs[id] # get the ID
    image_file = images["filename"][images["ID"]==curr_ID].unique() # get associated file for image
    
    image_ref = "!["+image_file+"](assets/"+image_file+")" # generate the markdown call for the image
    image_ref = image_ref[0]+"  "+  "\n"
    
    initials = image_file[0].split("_") # get initials for attribution
    attributions = [] # create a list of attributions
    for i in initials:
        if i=="A":
            attributions.append("Leaf traces by: Alejandra Rougon, UNAM ENES León, México  \n")
        elif i=="D":
            attributions.append("Leaf traces by: Dan Chitwood, Michigan State University, USA  \n")
        elif i=="K":
            attributions.append("Leaf traces by: Kylie DeViller, Acadia University, Canada  \n")
        elif i=="M":
            attributions.append("Leaf traces by: Mariana Ruíz Amaro, UNAM ENES León, México  \n")
        elif i=="N":
            attributions.append("Leaf traces by: Noé García, UNAM ENES León, México  \n")
        elif i=="Z":
            attributions.append("Leaf traces by: Zoë Migicovsky, Acadia University, Canada  \n")
    
    all_attr = "".join(attributions)

    # TEXTS
    # get unique subchapters for current ID
    subchapters = np.unique(texts["subchapter"][texts["ID"]==curr_ID].values)
    
    # for each subchapter, get associated hyperlinked texts
    curr_texts = []
    for i in range(len(subchapters)):
        path = "hyperlinked_text/"+subchapters[i].lstrip("0")+".txt"
        with open(path, 'r') as file:
            text = file.read()
        curr_texts.append(text)
    
    all_texts = "".join(curr_texts)

    # SYNONYMS
    
    # get synonyms for current ID
    syns = np.unique(texts["spelling"][texts["ID"]==curr_ID].values)
    for_syns = ["Variants: "]
    for syn in syns:
        for_syns.append(syn)
        for_syns.append(", ")
        
    syn_text = "".join(for_syns)[:-2]+"  \n\n"

    # PUT TOGETHER AND SAVE
    
    # get official name

    if all_texts=="":
        name = images["official_name"][images["ID"]==curr_ID].unique()[0] # get official name from images if no text
    else:
        name = texts["official_name"][texts["ID"]==curr_ID].unique()[0] # get official name
    
    # put it all together
    final_text = syn_text + all_texts + image_ref + all_attr
    
    # save
    path = folder_name + "/" + name + ".md"
    with open(path, "w", encoding="utf-8") as f:
        f.write(final_text) # just text

generate_mkdocs_nav(folder_name, output_file=folder_name+"_nav.txt")


Nav file written to: plants_with_illlustrations_nav.txt


# Plants (no illustrations)

In [229]:
# create a new folder to store chapter documents
folder_name = "plants"  
os.makedirs(folder_name, exist_ok=True)

# find IDs for plants with no images
plants = texts["ID"][texts["section"]=="plant"].unique()

# for each plant with no illustration ID
for id in range(len(plants)):

    curr_ID = plants[id] # get the ID

    # TEXTS
    # get unique subchapters for current ID
    subchapters = np.unique(texts["subchapter"][texts["ID"]==curr_ID].values)
    
    # for each subchapter, get associated hyperlinked texts
    curr_texts = []
    for i in range(len(subchapters)):
        path = "hyperlinked_text/"+subchapters[i].lstrip("0")+".txt"
        with open(path, 'r') as file:
            text = file.read()
        curr_texts.append(text)
    
    all_texts = "".join(curr_texts)

    # SYNONYMS
    
    # get synonyms for current ID
    syns = np.unique(texts["spelling"][texts["ID"]==curr_ID].values)
    for_syns = ["Variants: "]
    for syn in syns:
        for_syns.append(syn)
        for_syns.append(", ")
        
    syn_text = "".join(for_syns)[:-2]+"  \n\n"

    # PUT TOGETHER AND SAVE
    
    # get official name
    name = texts["official_name"][texts["ID"]==curr_ID].unique()[0] # get official name
    
    # put it all together
    final_text = syn_text + all_texts + image_ref + all_attr
    
    # save
    path = folder_name + "/" + name + ".md"
    with open(path, "w", encoding="utf-8") as f:
        f.write(final_text) # just text

generate_mkdocs_nav(folder_name, output_file=folder_name+"_nav.txt")

Nav file written to: plants_nav.txt


# Stones

In [230]:
# create a new folder to store chapter documents
folder_name = "stones"  
os.makedirs(folder_name, exist_ok=True)

# find IDs for stones
stones = texts["ID"][texts["section"]=="stone"].unique()

# for each stone
for id in range(len(stones)):

    curr_ID = stones[id] # get the ID

    # TEXTS
    # get unique subchapters for current ID
    subchapters = np.unique(texts["subchapter"][texts["ID"]==curr_ID].values)
    
    # for each subchapter, get associated hyperlinked texts
    curr_texts = []
    for i in range(len(subchapters)):
        path = "hyperlinked_text/"+subchapters[i].lstrip("0")+".txt"
        with open(path, 'r') as file:
            text = file.read()
        curr_texts.append(text)
    
    all_texts = "".join(curr_texts)

    # SYNONYMS
    
    # get synonyms for current ID
    syns = np.unique(texts["spelling"][texts["ID"]==curr_ID].values)
    for_syns = ["Variants: "]
    for syn in syns:
        for_syns.append(syn)
        for_syns.append(", ")
        
    syn_text = "".join(for_syns)[:-2]+"  \n\n"

    # PUT TOGETHER AND SAVE
    
    # get official name
    name = texts["official_name"][texts["ID"]==curr_ID].unique()[0] # get official name
    
    # put it all together
    final_text = syn_text + all_texts + image_ref + all_attr
    
    # save
    path = folder_name + "/" + name + ".md"
    with open(path, "w", encoding="utf-8") as f:
        f.write(final_text) # just text

generate_mkdocs_nav(folder_name, output_file=folder_name+"_nav.txt")

Nav file written to: stones_nav.txt


# Animals

In [231]:
# create a new folder to store chapter documents
folder_name = "animals"  
os.makedirs(folder_name, exist_ok=True)

# find IDs for animals 
animals = texts["ID"][texts["section"]=="animal"].unique()

# for each animal
for id in range(len(animals)):

    curr_ID = animals[id] # get the ID

    # TEXTS
    # get unique subchapters for current ID
    subchapters = np.unique(texts["subchapter"][texts["ID"]==curr_ID].values)
    
    # for each subchapter, get associated hyperlinked texts
    curr_texts = []
    for i in range(len(subchapters)):
        path = "hyperlinked_text/"+subchapters[i].lstrip("0")+".txt"
        with open(path, 'r') as file:
            text = file.read()
        curr_texts.append(text)
    
    all_texts = "".join(curr_texts)

    # SYNONYMS
    
    # get synonyms for current ID
    syns = np.unique(texts["spelling"][texts["ID"]==curr_ID].values)
    for_syns = ["Variants: "]
    for syn in syns:
        for_syns.append(syn)
        for_syns.append(", ")
        
    syn_text = "".join(for_syns)[:-2]+"  \n\n"

    # PUT TOGETHER AND SAVE
    
    # get official name
    name = texts["official_name"][texts["ID"]==curr_ID].unique()[0] # get official name
    
    # put it all together
    final_text = syn_text + all_texts + image_ref + all_attr
    
    # save
    path = folder_name + "/" + name + ".md"
    with open(path, "w", encoding="utf-8") as f:
        f.write(final_text) # just text

generate_mkdocs_nav(folder_name, output_file=folder_name+"_nav.txt")

Nav file written to: animals_nav.txt


# Birds

In [233]:
# create a new folder to store chapter documents
folder_name = "birds"  
os.makedirs(folder_name, exist_ok=True)

# find IDs for birds
birds = texts["ID"][texts["section"]=="bird"].unique()

# for each bird
for id in range(len(birds)):

    curr_ID = birds[id] # get the ID

    # TEXTS
    # get unique subchapters for current ID
    subchapters = np.unique(texts["subchapter"][texts["ID"]==curr_ID].values)
    
    # for each subchapter, get associated hyperlinked texts
    curr_texts = []
    for i in range(len(subchapters)):
        path = "hyperlinked_text/"+subchapters[i].lstrip("0")+".txt"
        with open(path, 'r') as file:
            text = file.read()
        curr_texts.append(text)
    
    all_texts = "".join(curr_texts)

    # SYNONYMS
    
    # get synonyms for current ID
    syns = np.unique(texts["spelling"][texts["ID"]==curr_ID].values)
    for_syns = ["Variants: "]
    for syn in syns:
        for_syns.append(syn)
        for_syns.append(", ")
        
    syn_text = "".join(for_syns)[:-2]+"  \n\n"

    # PUT TOGETHER AND SAVE
    
    # get official name
    name = texts["official_name"][texts["ID"]==curr_ID].unique()[0] # get official name
    
    # put it all together
    final_text = syn_text + all_texts + image_ref + all_attr
    
    # save
    path = folder_name + "/" + name + ".md"
    with open(path, "w", encoding="utf-8") as f:
        f.write(final_text) # just text

generate_mkdocs_nav(folder_name, output_file=folder_name+"_nav.txt")

Nav file written to: birds_nav.txt


# Other

In [234]:
# create a new folder to store chapter documents
folder_name = "other"  
os.makedirs(folder_name, exist_ok=True)

# find IDs for other
other = texts["ID"][texts["section"]=="other"].unique()

# for each other
for id in range(len(other)):

    curr_ID = other[id] # get the ID

    # TEXTS
    # get unique subchapters for current ID
    subchapters = np.unique(texts["subchapter"][texts["ID"]==curr_ID].values)
    
    # for each subchapter, get associated hyperlinked texts
    curr_texts = []
    for i in range(len(subchapters)):
        path = "hyperlinked_text/"+subchapters[i].lstrip("0")+".txt"
        with open(path, 'r') as file:
            text = file.read()
        curr_texts.append(text)
    
    all_texts = "".join(curr_texts)

    # SYNONYMS
    
    # get synonyms for current ID
    syns = np.unique(texts["spelling"][texts["ID"]==curr_ID].values)
    for_syns = ["Variants: "]
    for syn in syns:
        for_syns.append(syn)
        for_syns.append(", ")
        
    syn_text = "".join(for_syns)[:-2]+"  \n\n"

    # PUT TOGETHER AND SAVE
    
    # get official name
    name = texts["official_name"][texts["ID"]==curr_ID].unique()[0] # get official name
    
    # put it all together
    final_text = syn_text + all_texts + image_ref + all_attr
    
    # save
    path = folder_name + "/" + name + ".md"
    with open(path, "w", encoding="utf-8") as f:
        f.write(final_text) # just text

generate_mkdocs_nav(folder_name, output_file=folder_name+"_nav.txt")

Nav file written to: other_nav.txt
